<a href="https://colab.research.google.com/github/Vipriya24/Entity.co/blob/main/Market_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/Market_Sales.csv')
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [ ]:
print(df.isnull().sum())

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


In [ ]:
df['Item_Weight'] = df['Item_Weight'].fillna(df['Item_Weight'].mean())
df['Outlet_Size'] = df['Outlet_Size'].fillna('Unknown')

In [ ]:
label_encoder = LabelEncoder()
df['Outlet_Size'] = label_encoder.fit_transform(df['Outlet_Size'])
df['Item_Fat_Content'] = label_encoder.fit_transform(df['Item_Fat_Content'])
df['Outlet_Type'] = label_encoder.fit_transform(df['Outlet_Type'])

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_cols)

Categorical columns: ['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type']


In [ ]:
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Type,Item_Outlet_Sales,Item_Identifier_DRA24,Item_Identifier_DRA59,...,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3
0,9.300,1,0.016047,249.8092,1999,1,1,3735.1380,False,False,...,False,False,False,False,False,False,False,True,False,False
1,5.920,2,0.019278,48.2692,2009,1,2,443.4228,False,False,...,False,True,False,False,False,False,False,False,False,True
2,17.500,1,0.016760,141.6180,1999,1,1,2097.2700,False,False,...,False,False,False,False,False,False,False,True,False,False
3,19.200,2,0.000000,182.0950,1998,3,0,732.3800,False,False,...,False,False,False,False,False,False,False,False,False,True
4,8.930,1,0.000000,53.8614,1987,0,1,994.7052,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,1,0.056783,214.5218,1987,0,1,2778.3834,False,False,...,False,False,False,False,False,False,False,False,False,True
8519,8.380,2,0.046982,108.1570,2002,3,1,549.2850,False,False,...,False,False,False,False,False,True,False,False,True,False
8520,10.600,1,0.035186,85.1224,2004,2,1,1193.1136,False,False,...,False,False,False,False,True,False,False,False,True,False
8521,7.210,2,0.145221,103.1332,2009,1,2,1845.5976,False,False,...,False,True,False,False,False,False,False,False,False,True


### **Machine Learning WorkFlow**

In [ ]:
X = df.drop(['Item_Outlet_Sales'], axis=1)
y = df['Item_Outlet_Sales']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
print("Missing values in training data:")
print(X_train.isnull().sum())
print(y_train.isnull().sum())

Missing values in training data:
Item_Weight                    0
Item_Fat_Content               0
Item_Visibility                0
Item_MRP                       0
Outlet_Establishment_Year      0
                              ..
Outlet_Identifier_OUT045       0
Outlet_Identifier_OUT046       0
Outlet_Identifier_OUT049       0
Outlet_Location_Type_Tier 2    0
Outlet_Location_Type_Tier 3    0
Length: 1591, dtype: int64
0


In [ ]:
print("Data types in training data:")
print(X_train.dtypes)

Data types in training data:
Item_Weight                    float64
Item_Fat_Content                 int64
Item_Visibility                float64
Item_MRP                       float64
Outlet_Establishment_Year        int64
                                ...   
Outlet_Identifier_OUT045          bool
Outlet_Identifier_OUT046          bool
Outlet_Identifier_OUT049          bool
Outlet_Location_Type_Tier 2       bool
Outlet_Location_Type_Tier 3       bool
Length: 1591, dtype: object


### **XGBoost Algorithm**

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
predictions = model.predict(X_valid)

In [ ]:
print("Predicted Sales:", predictions)

Predicted Sales: [ 1.27018030e+03  6.94709167e+02  6.87804443e+02  4.60460059e+03
  2.10651709e+03  6.17041870e+02  5.28245117e+03  1.80286792e+03
  1.33424390e+03  2.88559082e+03  2.46283615e+02  6.40196533e+02
  6.13276367e+03  7.31197083e+02  8.30763672e+02  4.93334521e+03
  1.81635071e+03  1.73393823e+03  2.05063623e+03  3.26607178e+03
  3.07974219e+03  2.85486060e+03  3.32253784e+03  2.43561963e+03
  2.96801807e+03  1.42218713e+03  7.78395752e+02  3.86352563e+03
  3.08287573e+03  3.92550391e+03  7.64554565e+02  2.83399097e+03
  2.20881006e+03  3.64467603e+03  3.47902588e+02  1.27018030e+03
  7.27584961e+02  2.14255566e+03  1.25612744e+03  1.81956812e+03
  2.25464432e+02  2.15188135e+03  1.43884888e+03  1.45462036e+03
  2.51463306e+03  1.38131775e+03  8.00698792e+02  9.01424744e+02
  4.06691626e+03  2.07398438e+03  8.06199097e+02  1.95469092e+03
  9.25675049e+02  2.91638672e+02  6.87024780e+02  1.81635071e+03
  3.07041650e+03  2.07238745e+03  6.72267090e+02  4.10239307e+03
  8.8180

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Train-test split (90-10)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

# Feature Importance Analysis
temp_model = XGBRegressor(random_state=42)
temp_model.fit(X_train, y_train)
feature_importance = pd.Series(temp_model.feature_importances_, index=X_train.columns)
important_features = feature_importance[feature_importance > 0.01].index  # Keep only important features

X_train = X_train[important_features]
X_valid = X_valid[important_features]

# Hyperparameter Tuning with GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'n_estimators': [200, 500, 1000],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

xgb_model = XGBRegressor(random_state=42)

grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best Model
best_model = grid_search.best_estimator_

# Predict and Evaluate
predictions = best_model.predict(X_valid)
mse = mean_squared_error(y_valid, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_valid, predictions)

print("Optimized RMSE:", rmse)
print("Optimized R² Score:", r2)

Optimized RMSE: 1026.212730099671
Optimized R² Score: 0.6233306539654524


In [ ]:
predicted_df = pd.DataFrame({'Actual': y_valid, 'Predicted': predictions})
predicted_df.to_csv('predicted_sales_xgboost.csv', index=False)